In [1]:
from src.common.vector import *
from src.common.reference_frame import *

r = Vector(-5368,-1784,3691)
(ra,dec) = radec_from_r(r)

print(f'RA = {np.rad2deg(ra)}\nDEC = {np.rad2deg(dec)}')

RA = 198.38370037548617
DEC = 33.12454287112769


In [2]:
from src.common.vector import *
from src.common.reference_frame import *
from src.common.time import *
from datetime import datetime

date_1 = datetime(2004,5,12,14,45,30)
jd_1 = julian_day(date_1)
date_2 = datetime(1957,10,4,19,26,24)
jd_2 = julian_day(date_2)
print(f'JD_1 = {jd_1} dias\nJD_2 = {jd_2} dias')
print(f'Diferença = {jd_1 - jd_2} dias')


JD_1 = 2453138.1149305557 dias
JD_2 = 2436116.31 dias
Diferença = 17021.804930555634 dias


In [3]:
from src.common.vector import *
from src.common.time import *
from datetime import datetime

theta = sidereal_time(date=datetime(2004,3,3,4,30,0),longitude=139.8)
theta

8.593542535242534

In [4]:
# from src.common.vector import *
# from src.common.reference_frame import *
# import math

# r = Vector(x=-5368,y=-1784,z=3691)
# r_o = get_topocentric_origin(60,20,0,theta_g=126.7)
# print(f'r = {round(r_o.x,0)}I + ({round(r_o.y,1)})J + ({round(r_o.z,0)})K')
# rho = r-r_o
# ra,dec = radec_from_r(rho)
# print(f'RA = {round(np.rad2deg(ra),1)}°\nDEC = {round(np.rad2deg(dec),2)}°')

r = -5959.0I + (-700.1)J + (2155.0)K
RA = 298.6°
DEC = 51.21°


In [33]:
# Observatório da Aldoria
lat = -30.5262693
lon = -70.8531799
alt = 2180e-3
date = datetime(2023,4,25,9,50,32)
r_origin = get_topocentric_origin(latitude=lat,longitude=lon,altitude=alt,date=date)

ra_object = np.deg2rad(204.6559)
dec_object = np.deg2rad(-45.703182)

rho = Vector(x = 50000*np.cos(dec_object)*np.cos(ra_object),y=50000*np.cos(dec_object)*np.sin(ra_object),z = 50000*np.sin(dec_object))

r_obj = r_origin + rho

ra,dec = radec_from_r(r_obj)


print(f'r = {round(r_origin.x,0)}I + ({round(r_origin.y,1)})J + ({round(r_origin.z,0)})K')
print(f'RA = {round(np.rad2deg(ra),1)}°\nDEC = {round(np.rad2deg(dec),2)}°')

r = 1879.0I + (-5179.8)J + (-3206.0)K
RA = 213.5°
DEC = -47.45°


In [27]:
print(r_origin)

x = 1879.0831301397955
y = -5179.750714959119
z = -3205.7270852487895
Magnitude = 6374.751526466987
Theta_x = 72.85627627903577
Theta_y = 144.34499973606847
Theta_z = 120.19064009272486
